In [3]:
!pip install tiktoken

In [4]:
from importlib.metadata import version

print("torch version:", version("torch"))
print("tiktoken version:", version("tiktoken"))

torch version: 2.1.2+cpu
tiktoken version: 0.7.0


In [5]:
import urllib

# URL of the PDF file
url = "https://ag.gov.np/files/Constitution-of-Nepal_2072_Eng_www.moljpa.gov_.npDate-72_11_16.pdf"

# Open the URL and download the PDF
response = urllib.request.urlopen(url)
pdf_data = response.read()

# Save the PDF to a local file
with open("Constitution-of-Nepal_2072_Eng.pdf", "wb") as f:
    f.write(pdf_data)

print("PDF downloaded and saved as 'Constitution-of-Nepal_2072_Eng.pdf'")


PDF downloaded and saved as 'Constitution-of-Nepal_2072_Eng.pdf'


In [6]:
!pip install PyPDF2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 6.9 MB/s eta 0:00:00


In [7]:
import PyPDF2

# Open the downloaded PDF file
with open("Constitution-of-Nepal_2072_Eng.pdf", "rb") as file:
    # Initialize a PDF reader object
    pdf_reader = PyPDF2.PdfReader(file)
    
    # Extract text from each page of the PDF
    raw_text = ""
    for page_num in range(len(pdf_reader.pages)):
        page = pdf_reader.pages[page_num]
        raw_text += page.extract_text()

# Print the first 500 characters to preview the content
print(raw_text[:50])


 
1 
  
 
 
 
THE CONSTITUTION OF NEPAL  
   
2 
 


In [8]:
print(len(raw_text))

344782


**Split text into tokens**

In [9]:
import re
preprocessed = re.split(r'([,.:;?_!-"()\']|--|\s|[@#%&])', raw_text)

preprocessed = [item.strip() for item in preprocessed if item.strip()]
print(preprocessed[:30])

['1', 'THE', 'CONSTITUTION', 'OF', 'NEPAL', '2', 'Table', 'of', 'Content', 's', 'Preamble', 'Part-1', 'Preliminary', 'Part-2', 'Citizenship', 'Part-3', 'Fundamental', 'Rights', 'and', 'Duties', 'Part-4', 'Directive', 'Principles', ',', 'Policies', 'and', 'Obligations', 'of', 'the', 'State']


In [10]:
print(len(preprocessed))

65101


In [11]:
#Unique tokens
len(set(preprocessed))

4525

**Create a vocabulary**

In [12]:
all_words = sorted(set(preprocessed))
vocab_size = len(all_words)

print(vocab_size)

4525


In [13]:
#Give indexes and put that as dictionary
vocab = {token:integer for integer,token in enumerate(all_words)}

In [17]:
"""Putting all together"""

class SimpleTokenizerV1:
    def __init__(self, vocab):
        self.str_to_int = vocab
        self.int_to_str = {i:s for s,i in vocab.items()}
    
    def encode(self, text):
        preprocessed = re.split(r'([,.:;?_!"()\']|--|\s)', text)
                                
        preprocessed = [
            item.strip() for item in preprocessed if item.strip()
        ]
        ids = [self.str_to_int[s] for s in preprocessed]
        return ids
        
    def decode(self, ids):
        text = " ".join([self.int_to_str[i] for i in ids])
        # Replace spaces before the specified punctuations
        text = re.sub(r'\s+([,.?!"()\'])', r'\1', text)
        return text

In [24]:
tokenizer = SimpleTokenizerV1(vocab)
tokenizer.encode(""" """)


[]

**Byte Pair Encoding**

In [25]:
import importlib
import tiktoken

print("tiktoken version:", importlib.metadata.version("tiktoken"))

tiktoken version: 0.7.0


In [26]:
tokenizer = tiktoken.get_encoding("gpt2")

In [27]:
text = (
    "No law shall be made providing for the death penalty to any one<|endoftext|> Every citizen"
     "of"
)

integers = tokenizer.encode(text, allowed_special={"<|endoftext|>"})

print(integers)

[2949, 1099, 2236, 307, 925, 4955, 329, 262, 1918, 7389, 284, 597, 530, 50256, 3887, 9511, 1659]


In [28]:
tokenizer.encode("fddfsdfsf")

[69, 1860, 9501, 7568, 28202]

In [32]:
tokenizer.decode([1860])

'dd'

**Data Sampling with a sliding window**

**We train LLMs to generate one word at a time, so we want to prepare the training data accordingly where the next word in a sequence represents the target to predict**

In [33]:
import PyPDF2
# Open the downloaded PDF file
with open("Constitution-of-Nepal_2072_Eng.pdf", "rb") as file:
    # Initialize a PDF reader object
    pdf_reader = PyPDF2.PdfReader(file)
    
    # Extract text from each page of the PDF
    raw_text = ""
    for page_num in range(len(pdf_reader.pages)):
        page = pdf_reader.pages[page_num]
        raw_text += page.extract_text()
enc_text = tokenizer.encode(raw_text)
print(len(enc_text))

86531
